In [1]:
# Análisis y manejo de datos
import pandas as pd 
import numpy as np 

#Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

print("Librerías correctamente importadas")

Librerías correctamente importadas


In [5]:
df = pd.read_csv('data.csv')
print('Preview del dataset:')
display(df.head(5))
print('Valores nulos')
print(df.isnull().sum())
print('Apariciones de cada estado')
print(df['ENTIDAD'].value_counts())

Preview del dataset:


,PERIODO,CVE_ENT,ENTIDAD,MINIMA,MEDIA,MAXIMA,PRECIPITACION
0,1985-01-01,0,Nacional,7.8,15.9,23.9,36.0
1,1985-01-01,1,Aguascalientes,3.1,12.2,21.3,4.9
2,1985-01-01,2,Baja California,5.6,12.9,20.2,12.2
3,1985-01-01,3,Baja California Sur,9.2,17.1,25.0,30.3
4,1985-01-01,4,Campeche,15.7,22.7,29.7,20.9


Valores nulos
PERIODO          0
CVE_ENT          0
ENTIDAD          0
MINIMA           0
MEDIA            0
MAXIMA           0
PRECIPITACION    0
dtype: int64
Apariciones de cada estado
ENTIDAD
Nacional               486
Aguascalientes         486
Baja California        486
Baja California Sur    486
Campeche               486
Coahuila               486
Colima                 486
Chiapas                486
Chihuahua              486
Ciudad de México       486
Durango                486
Guanajuato             486
Guerrero               486
Hidalgo                486
Jalisco                486
Estado de México       486
Michoacán              486
Morelos                486
Nayarit                486
Nuevo León             486
Oaxaca                 486
Puebla                 486
Querétaro              486
Quintana Roo           486
San Luis Potosí        486
Sinaloa                486
Sonora                 486
Tabasco                486
Tamaulipas             486
Tlaxcala              